## WebScraping con requests y BeautifulSoup

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [26]:
url = "https://fbref.com/en/comps/31/Liga-MX-Stats"

In [27]:
data = requests.get(url)
data

<Response [200]>

In [28]:
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202403072" />\n<!-- InMobi Choice. Consent Manager Tag v3.0 (for TCF 2.2) -->\n<script type="text/javascript" async=true>\n(function() {\n  var host = window.location.hostname;\n  var element = document.createElement(\'script\');\n  var firstScript = document.getElementsByTagName(\'script\')[0];\n  var url = \'https://cmp.inmobi.com\'\n    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \'/choice.js?tag_version=V3\');\n  var uspTries = 0;\n  var uspTriesLimit = 3;\n  element.async = true;\n  element.type = \'text/javascript\';\n  element.src = url;\n\n  firstScript.parentNode.insertBefore(element, firstScript

In [29]:
soup = BeautifulSoup(data.text)

In [30]:
#Obtenemos tabla de donde sacaremos los datos
standings_table = soup.select('table.stats_table')[0]
standings_table

<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2023-2024310_overall"> <caption>League Table Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">Rk</th> <th aria-label="Squad" class="poptip sort_default_asc center" data-stat="team" scope="col">Squad</th> <th aria-label="Matc

In [31]:
#Se guardara en links todas las etiquetas 'a' con links hacia cada equipo encontrado ahi
links = standings_table.find_all('a')

In [32]:
'''Devolvera algunos datos que no necesitamos
ej. Jugadores y Partidos, que no utilizaremos ahora'''

links = [l.get("href") for l in links]

In [33]:
# Filtra para obtener solo los links de los equipos
links = [l for l in links if '/squads/' in l]
links

['/en/squads/18d3c3a3/America-Stats',
 '/en/squads/dd5ca9bd/Monterrey-Stats',
 '/en/squads/d9e1bd51/UANL-Stats',
 '/en/squads/1be8d2e3/Pachuca-Stats',
 '/en/squads/c9d59c6c/Pumas-UNAM-Stats',
 '/en/squads/c02b0f7a/Guadalajara-Stats',
 '/en/squads/44b88a4e/Toluca-Stats',
 '/en/squads/632f1838/Cruz-Azul-Stats',
 '/en/squads/fd7dad55/Leon-Stats',
 '/en/squads/5d274ee4/Atletico-Stats',
 '/en/squads/03b65ba9/Santos-Laguna-Stats',
 '/en/squads/f0297c23/Mazatlan-Stats',
 '/en/squads/752db496/Necaxa-Stats',
 '/en/squads/c3352ce7/Queretaro-Stats',
 '/en/squads/73fd2313/Puebla-Stats',
 '/en/squads/a42ddf2f/Tijuana-Stats',
 '/en/squads/7c76bc53/Atlas-Stats',
 '/en/squads/29bff345/FC-Juarez-Stats']

In [34]:
#Convertir a url
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/18d3c3a3/America-Stats',
 'https://fbref.com/en/squads/dd5ca9bd/Monterrey-Stats',
 'https://fbref.com/en/squads/d9e1bd51/UANL-Stats',
 'https://fbref.com/en/squads/1be8d2e3/Pachuca-Stats',
 'https://fbref.com/en/squads/c9d59c6c/Pumas-UNAM-Stats',
 'https://fbref.com/en/squads/c02b0f7a/Guadalajara-Stats',
 'https://fbref.com/en/squads/44b88a4e/Toluca-Stats',
 'https://fbref.com/en/squads/632f1838/Cruz-Azul-Stats',
 'https://fbref.com/en/squads/fd7dad55/Leon-Stats',
 'https://fbref.com/en/squads/5d274ee4/Atletico-Stats',
 'https://fbref.com/en/squads/03b65ba9/Santos-Laguna-Stats',
 'https://fbref.com/en/squads/f0297c23/Mazatlan-Stats',
 'https://fbref.com/en/squads/752db496/Necaxa-Stats',
 'https://fbref.com/en/squads/c3352ce7/Queretaro-Stats',
 'https://fbref.com/en/squads/73fd2313/Puebla-Stats',
 'https://fbref.com/en/squads/a42ddf2f/Tijuana-Stats',
 'https://fbref.com/en/squads/7c76bc53/Atlas-Stats',
 'https://fbref.com/en/squads/29bff345/FC-Juarez-Stats'

## Extraer estadísticas de los partidos usando pandas and requests

In [35]:
#En esta parte solo usaremos el 1er equipo para entender mejor
fteam_url = team_urls[0]
fteam_url

'https://fbref.com/en/squads/18d3c3a3/America-Stats'

In [36]:
data = requests.get(fteam_url)
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202403072" />\n<!-- InMobi Choice. Consent Manager Tag v3.0 (for TCF 2.2) -->\n<script type="text/javascript" async=true>\n(function() {\n  var host = window.location.hostname;\n  var element = document.createElement(\'script\');\n  var firstScript = document.getElementsByTagName(\'script\')[0];\n  var url = \'https://cmp.inmobi.com\'\n    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \'/choice.js?tag_version=V3\');\n  var uspTries = 0;\n  var uspTriesLimit = 3;\n  element.async = true;\n  element.type = \'text/javascript\';\n  element.src = url;\n\n  firstScript.parentNode.insertBefore(element, firstScript

In [37]:
matches = pd.read_html(data.text, match ="Scores & Fixtures")
matches[0]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-06-30,20:00,Liga MX,Apertura 2023 Regular Season,Fri,Home,L,1.0,2.0,FC Juárez,0.9,0.7,61.0,13391.0,Miguel Layún,4-2-3-1,Ismael Lopez,Match Report,NaN
1,2023-07-15,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,W,3.0,0.0,Puebla,3.6,0.9,60.0,32294.0,Miguel Layún,4-2-3-1,Víctor Cáceres,Match Report,NaN
2,2023-08-20,22:10,Liga MX,Apertura 2023 Regular Season,Sun,Home,D,1.0,1.0,Atlas,1.8,0.9,64.0,17046.0,Diego Valdés,4-2-3-1,Adonai Escobedo,Match Report,NaN
3,2023-08-23,20:00,Liga MX,Apertura 2023 Regular Season,Wed,Home,W,3.0,2.0,Necaxa,2.4,0.6,70.0,19085.0,Álvaro Fidalgo,4-2-3-1,Marco Ortíz,Match Report,NaN
4,2023-08-26,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,D,1.0,1.0,León,1.6,0.7,54.0,30701.0,Jonathan dos Santos,4-2-3-1,Óscar Macias,Match Report,NaN
5,2023-09-02,22:10,Liga MX,Apertura 2023 Regular Season,Sat,Away,W,3.0,2.0,Cruz Azul,0.6,1.2,48.0,36500.0,Jonathan dos Santos,4-2-3-1,Daniel Quintero,Match Report,NaN
6,2023-09-16,22:10,Liga MX,Apertura 2023 Regular Season,Sat,Home,W,4.0,0.0,Guadalajara,1.3,0.4,53.0,62514.0,Jonathan dos Santos,4-2-3-1,Fernando Guerrero,Match Report,NaN
7,2023-09-20,21:06,Liga MX,Apertura 2023 Regular Season,Wed,Away,W,2.0,1.0,Querétaro,1.0,0.8,67.0,31296.0,Diego Valdés,4-2-3-1,Diego Montaño,Match Report,NaN
8,2023-09-24,13:00,Liga MX,Apertura 2023 Regular Season,Sun,Away,D,1.0,1.0,Toluca,2.0,1.5,51.0,27273.0,Jonathan dos Santos,4-2-3-1,Adonai Escobedo,Match Report,NaN
9,2023-09-30,19:40,Liga MX,Apertura 2023 Regular Season,Sat,Home,W,1.0,0.0,UNAM,1.9,0.6,49.0,62707.0,Henry Martín,4-4-2,Víctor Cáceres,Match Report,NaN


## Obtener datos adicionales (estadisticas de tiros) con pandas y requests

In [38]:
soup = BeautifulSoup(data.text)

In [39]:
#Busca los tags 'a' y en ellos obtiene el link de estadisticas de tiro si se encuentran en esa seccion
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
links

['/en/squads/18d3c3a3/2023-2024/matchlogs/all_comps/shooting/America-Match-Logs-All-Competitions']

In [40]:
#Descargar el html de la pagina de estadistica de tiro 
data = requests.get(f"https://fbref.com{links[0]}")
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202403072" />\n<!-- InMobi Choice. Consent Manager Tag v3.0 (for TCF 2.2) -->\n<script type="text/javascript" async=true>\n(function() {\n  var host = window.location.hostname;\n  var element = document.createElement(\'script\');\n  var firstScript = document.getElementsByTagName(\'script\')[0];\n  var url = \'https://cmp.inmobi.com\'\n    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \'/choice.js?tag_version=V3\');\n  var uspTries = 0;\n  var uspTriesLimit = 3;\n  element.async = true;\n  element.type = \'text/javascript\';\n  element.src = url;\n\n  firstScript.parentNode.insertBefore(element, firstScript

In [41]:
#Obtenemos el html y agregamos un indice en 0 para que se convierta en una matriz (tabla)
shooting = pd.read_html(data.text, match = "Shooting")[0]
shooting.head()

For América                                                                  \
         Date   Time     Comp                         Round  Day Venue Result   
0  2023-06-30  20:00  Liga MX  Apertura 2023 Regular Season  Fri  Home      L   
1  2023-07-15  20:00  Liga MX  Apertura 2023 Regular Season  Sat  Home      W   
2  2023-08-20  22:10  Liga MX  Apertura 2023 Regular Season  Sun  Home      D   
3  2023-08-23  20:00  Liga MX  Apertura 2023 Regular Season  Wed  Home      W   
4  2023-08-26  20:00  Liga MX  Apertura 2023 Regular Season  Sat  Home      D   

                        ... Standard             Expected                    \
    GF   GA   Opponent  ...     Dist FK PK PKatt       xG npxG npxG/Sh G-xG   
0  1.0  2.0  FC Juárez  ...     21.8  2  0     0      0.9  0.9    0.06  0.1   
1  3.0  0.0     Puebla  ...     13.9  1  0     0      3.6  3.6    0.18 -0.6   
2  1.0  1.0      Atlas  ...     12.0  1  0     0      1.8  1.8    0.15 -0.8   
3  3.0  2.0     Necaxa  ...     14.5  0  0     0      2.4  2.4    0.12  0.6   
4  1.0  1.0       León  ...     18.8  0  1     1      1.6  0.8    0.09 -0.6   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     0.1        Match Report  
1    -0.6        Match Report  
2    -0.8        Match Report  
3     0.6        Match Report  
4    -0.8        Match Report  

[5 rows x 26 columns]

## Limpiar y unir datos obtenidos con pandas

In [42]:
#Hay un doble indice en la tabla, no es necesario para pandas, lo mejor es quitarlo 
shooting.columns = shooting.columns.droplevel()
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-06-30,20:00,Liga MX,Apertura 2023 Regular Season,Fri,Home,L,1.0,2.0,FC Juárez,...,21.8,2,0,0,0.9,0.9,0.06,0.1,0.1,Match Report
1,2023-07-15,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,W,3.0,0.0,Puebla,...,13.9,1,0,0,3.6,3.6,0.18,-0.6,-0.6,Match Report
2,2023-08-20,22:10,Liga MX,Apertura 2023 Regular Season,Sun,Home,D,1.0,1.0,Atlas,...,12.0,1,0,0,1.8,1.8,0.15,-0.8,-0.8,Match Report
3,2023-08-23,20:00,Liga MX,Apertura 2023 Regular Season,Wed,Home,W,3.0,2.0,Necaxa,...,14.5,0,0,0,2.4,2.4,0.12,0.6,0.6,Match Report
4,2023-08-26,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,D,1.0,1.0,León,...,18.8,0,1,1,1.6,0.8,0.09,-0.6,-0.8,Match Report


In [43]:
shooting["Date"]

0     2023-06-30
1     2023-07-15
2     2023-08-20
3     2023-08-23
4     2023-08-26
5     2023-09-02
6     2023-09-16
7     2023-09-20
8     2023-09-24
9     2023-09-30
10    2023-10-03
11    2023-10-06
12    2023-10-21
13    2023-10-28
14    2023-11-01
15    2023-11-04
16    2023-11-11
17    2023-11-29
18    2023-12-02
19    2023-12-06
20    2023-12-09
21    2023-12-14
22    2023-12-17
23    2024-01-13
24    2024-01-20
25    2024-01-24
26    2024-01-27
27    2024-02-03
28    2024-02-10
29    2024-02-17
30    2024-02-21
31    2024-02-24
32    2024-03-02
33           NaN
Name: Date, dtype: object

In [44]:
#Unimos ambos df, no completamente para evitar columnas duplicadas, por eso solo se eligen las que se ven abajo
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-06-30,20:00,Liga MX,Apertura 2023 Regular Season,Fri,Home,L,1.0,2.0,FC Juárez,...,4-2-3-1,Ismael Lopez,Match Report,NaN,14,3,21.8,2,0,0
1,2023-07-15,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,W,3.0,0.0,Puebla,...,4-2-3-1,Víctor Cáceres,Match Report,NaN,21,8,13.9,1,0,0
2,2023-08-20,22:10,Liga MX,Apertura 2023 Regular Season,Sun,Home,D,1.0,1.0,Atlas,...,4-2-3-1,Adonai Escobedo,Match Report,NaN,13,4,12.0,1,0,0
3,2023-08-23,20:00,Liga MX,Apertura 2023 Regular Season,Wed,Home,W,3.0,2.0,Necaxa,...,4-2-3-1,Marco Ortíz,Match Report,NaN,20,10,14.5,0,0,0
4,2023-08-26,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,D,1.0,1.0,León,...,4-2-3-1,Óscar Macias,Match Report,NaN,9,2,18.8,0,1,1


## Haremos el modelo mas grande, para todos los equipos, iterando para cada uno con ciclos

In [45]:
years = list(range(2024, 2020, -1))
years
print(years)  # This should print [2024, 2023, 2022, 2021, 2020]

[2024, 2023, 2022, 2021]


In [46]:
#Guardaremos todos los dataframes combinados de todos los equipos de la liga
all_matches = []

In [47]:
#Necesitamos usar para eso el mismo link de donde sacamos todo desde el inicio y ademas el link para las previous seasons y obtener las antiguas temporadas
standings_url = "https://fbref.com/en/comps/31/Liga-MX-Stats"

In [48]:
'''Loop que obtiene del link de la pagina el html, luego encuentra la tabla de estadisticas, y ahi busca tags <a> 
(contienen link hacia pag de cada equipo) a cada uno de esos links relativos les agrega el prefijo de la direccion'''
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    #url relativos -> url completos
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    
    '''Obtenemos el link que nos lleva hacia la temporada pasada haciendolo link completo'''
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    
    
    '''Esto obtiene el nombre del equipo desde la url, corta todo lo que este antes del ultimo slash, 
    remueve 'Stats' y quita guiones si es que hay '''
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match ="Scores & Fixtures")[0]
        
        '''Esta seccion obtiene la pagina de estadisticas de tiro de cada equipo'''
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match ="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        '''Como buena practica dejamos este error handler para los casos donde los equipos no tengan esas estadisticas'''
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
        except ValueError:
            continue
        
        '''Como todas las estadisticas se van a unir, crearemos columnas con el nombre del equipo correspondiente para evitar confusiones'''
        #team_data = team_data[team_data["Comp"] == "Liga MX"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(3)
        

In [52]:
len(all_matches)

72

In [50]:
#Agrega a la variable match_df todos los df que se creen con el ciclo de arriba
match_df = pd.concat(all_matches)

In [51]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2023-06-30,20:00,Liga MX,Apertura 2023 Regular Season,Fri,Home,L,1.0,2.0,FC Juárez,...,Match Report,NaN,14,3,21.8,2.0,0,0,2024,America
1,2023-07-15,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,W,3.0,0.0,Puebla,...,Match Report,NaN,21,8,13.9,1.0,0,0,2024,America
2,2023-08-20,22:10,Liga MX,Apertura 2023 Regular Season,Sun,Home,D,1.0,1.0,Atlas,...,Match Report,NaN,13,4,12.0,1.0,0,0,2024,America
3,2023-08-23,20:00,Liga MX,Apertura 2023 Regular Season,Wed,Home,W,3.0,2.0,Necaxa,...,Match Report,NaN,20,10,14.5,0.0,0,0,2024,America
4,2023-08-26,20:00,Liga MX,Apertura 2023 Regular Season,Sat,Home,D,1.0,1.0,León,...,Match Report,NaN,9,2,18.8,0.0,1,1,2024,America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,2021-04-03,21:06,Liga MX,Guardianes 2021 Regular Season,Sat,Away,L,0,2,Monterrey,...,Match Report,NaN,10,5,17.6,0.0,0,0,2021,Atletico
30,2021-04-09,20:30,Liga MX,Guardianes 2021 Regular Season,Fri,Away,L,1,2,FC Juárez,...,Match Report,NaN,12,4,18.5,0.0,1,1,2021,Atletico
31,2021-04-17,17:00,Liga MX,Guardianes 2021 Regular Season,Sat,Home,L,1,4,Puebla,...,Match Report,NaN,12,5,17.3,0.0,0,0,2021,Atletico
32,2021-04-24,17:00,Liga MX,Guardianes 2021 Regular Season,Sat,Away,L,2,3,Cruz Azul,...,Match Report,NaN,16,2,17.1,0.0,1,1,2021,Atletico


In [53]:
match_df.to_csv("matches.csv")